# 03: Comprehensive Fairness Analysis

This notebook provides a comprehensive evaluation of baseline models for group-level disparities:
- Demographic parity analysis across all protected attributes
- Equalized odds assessment (TPR and FPR)
- Precision parity evaluation
- Disaggregated performance metrics by protected attributes
- Fairlearn-based comprehensive analysis
- Visualizations of fairness metrics for all models and attributes


In [ ]:
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Install packages (for Google Colab)
if IN_COLAB:
    !pip install scikit-learn matplotlib seaborn fairlearn -q

# Imports
import pandas as pd
import numpy as np
import pickle
import json
from pathlib import Path
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, roc_curve
)
from fairlearn.metrics import (
    MetricFrame, demographic_parity_difference, demographic_parity_ratio,
    equalized_odds_difference, equalized_odds_ratio,
    false_positive_rate, false_negative_rate
)
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting setup
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

# Set random seed for reproducibility
np.random.seed(42)

print("=" * 60)
print("COMPREHENSIVE FAIRNESS ANALYSIS")
print("=" * 60)
print("✓ Setup complete!")


## 1. Load Data, Models, and Predictions

Load the preprocessed data, trained models, and predictions from previous notebooks.


In [ ]:
# Determine data directory (works for both local and Colab)
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    possible_paths = [
        Path('/content/drive/MyDrive/6.3950_project/processed_data'),
        Path('/content/drive/Shareddrives/Insurance Fairness/processed_data'),
        Path('/content/drive/MyDrive/insurance-fairness/results'),
        Path('../results')
    ]
else:
    possible_paths = [
        Path('../results'),
        Path('./results')
    ]

results_dir = None
for path in possible_paths:
    if path.exists() and (path / 'X_test.pkl').exists():
        results_dir = path
        break

if results_dir is None:
    error_msg = (
        "Could not find results directory. Please run notebooks 01 and 02 first.\n"
        f"Tried paths: {', '.join([str(p) for p in possible_paths])}"
    )
    raise FileNotFoundError(error_msg)

print(f"✓ Loading data from: {results_dir}")

# Load test data and protected attributes
with open(results_dir / 'X_test.pkl', 'rb') as f: X_test = pickle.load(f)
with open(results_dir / 'y_test.pkl', 'rb') as f: y_test = pickle.load(f)
with open(results_dir / 'protected_test.pkl', 'rb') as f: protected_test = pickle.load(f)

# Load models
print("Loading models...")
with open(results_dir / 'logistic_regression_model.pkl', 'rb') as f:
    lr_model = pickle.load(f)
with open(results_dir / 'random_forest_model.pkl', 'rb') as f:
    rf_model = pickle.load(f)

# Load predictions
print("Loading predictions...")
with open(results_dir / 'lr_test_predictions.pkl', 'rb') as f:
    lr_test_pred = pickle.load(f)
with open(results_dir / 'lr_test_proba.pkl', 'rb') as f:
    lr_test_proba = pickle.load(f)
with open(results_dir / 'rf_test_predictions.pkl', 'rb') as f:
    rf_test_pred = pickle.load(f)
with open(results_dir / 'rf_test_proba.pkl', 'rb') as f:
    rf_test_proba = pickle.load(f)

# Load metadata
try:
    with open(results_dir / 'preprocessing_metadata.json', 'r') as f:
        metadata = json.load(f)
    protected_attributes_list = metadata.get('protected_attributes', [])
except:
    # Fallback: infer from protected_test columns
    protected_attributes_list = protected_test.columns.tolist()

print(f"\n✅ Data loaded!")
print(f"   Test set: {len(X_test):,} samples")
print(f"   Protected attributes: {', '.join(protected_attributes_list)}")
print(f"   Class distribution: {(y_test == 0).sum():,} No, {(y_test == 1).sum():,} Yes")


## 2. Helper Functions for Fairness Analysis

Define reusable functions for calculating and visualizing fairness metrics.

### 2.1 Understanding Fairness Metrics

**Demographic Parity (Statistical Parity):**
- Measures equal selection rates across groups
- Formula: P(Ŷ=1 | A=a) should be equal for all groups
- Ideal difference: 0 (perfect parity)
- Threshold: Differences > 0.05 (5 percentage points) indicate significant disparities

**Equalized Odds:**
- Measures equal true positive rates (TPR) and false positive rates (FPR) across groups
- TPR: P(Ŷ=1 | Y=1, A=a) - equal across groups
- FPR: P(Ŷ=1 | Y=0, A=a) - equal across groups
- Ideal difference: 0 for both TPR and FPR
- Ensures model performs equally well for all groups

**Precision Parity:**
- Measures equal precision (positive predictive value) across groups
- Formula: P(Y=1 | Ŷ=1, A=a) should be equal
- Ideal difference: 0
- Ensures equal confidence in positive predictions across groups

**Interpretation:**
- Lower differences indicate better fairness
- Differences > 0.05 (5pp) are flagged as significant disparities
- All metrics are calculated on the test set for unbiased evaluation


In [ ]:
def calculate_fairness_metrics(y_true, y_pred, sensitive_feature, model_name, attr_name):
    """
    Calculate comprehensive fairness metrics for a given model and protected attribute.
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred : array-like
        Predicted labels
    sensitive_feature : array-like
        Protected attribute values
    model_name : str
        Name of the model
    attr_name : str
        Name of the protected attribute
        
    Returns:
    --------
    dict : Dictionary containing all fairness metrics
    """
    # Use fairlearn MetricFrame for comprehensive metrics
    metrics_dict = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'f1': f1_score,
        'false_positive_rate': false_positive_rate,
        'false_negative_rate': false_negative_rate
    }
    
    metric_frame = MetricFrame(
        metrics=metrics_dict,
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_feature
    )
    
    # Calculate group-level metrics
    by_group = metric_frame.by_group
    
    # Calculate fairness metrics
    dp_diff = demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_feature)
    dp_ratio = demographic_parity_ratio(y_true, y_pred, sensitive_features=sensitive_feature)
    eo_diff = equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_feature)
    eo_ratio = equalized_odds_ratio(y_true, y_pred, sensitive_features=sensitive_feature)
    
    # Calculate precision parity (precision difference across groups)
    precision_by_group = by_group['precision']
    precision_diff = precision_by_group.max() - precision_by_group.min()
    precision_ratio = precision_by_group.min() / precision_by_group.max() if precision_by_group.max() > 0 else 0
    
    # Calculate selection rate (approval rate) by group
    # Also validate group sizes
    selection_by_group = {}
    group_sizes = {}
    min_group_size = 10  # Minimum samples per group for reliable metrics
    
    for group in np.unique(sensitive_feature):
        mask = sensitive_feature == group
        group_size = mask.sum()
        group_sizes[group] = group_size
        
        if group_size < min_group_size:
            print(f"   ⚠ Warning: Group '{group}' has only {group_size} samples (< {min_group_size})")
        
        selection_by_group[group] = y_pred[mask].mean()
    
    # Calculate TPR and FPR differences
    tpr_by_group = by_group['recall']  # TPR = Recall
    fpr_by_group = by_group['false_positive_rate']
    tpr_diff = tpr_by_group.max() - tpr_by_group.min()
    fpr_diff = fpr_by_group.max() - fpr_by_group.min()
    
    return {
        'model_name': model_name,
        'attribute_name': attr_name,
        'by_group': by_group,
        'demographic_parity_difference': dp_diff,
        'demographic_parity_ratio': dp_ratio,
        'equalized_odds_difference': eo_diff,
        'equalized_odds_ratio': eo_ratio,
        'precision_difference': precision_diff,
        'precision_ratio': precision_ratio,
        'tpr_difference': tpr_diff,
        'fpr_difference': fpr_diff,
        'selection_by_group': selection_by_group,
        'tpr_by_group': tpr_by_group.to_dict(),
        'fpr_by_group': fpr_by_group.to_dict(),
        'precision_by_group': precision_by_group.to_dict(),
        'group_sizes': group_sizes
    }


def visualize_fairness_metrics(metrics_dict, model_name, attr_name):
    """
    Create visualizations for fairness metrics.
    """
    by_group = metrics_dict['by_group']
    # Ensure groups match selection_by_group keys and are sorted for consistency
    groups = sorted(list(metrics_dict['selection_by_group'].keys()))
    
    # Verify all groups exist in by_group
    missing_groups = [g for g in groups if g not in by_group.index]
    if missing_groups:
        print(f"⚠ Warning: Groups {missing_groups} not found in by_group, skipping visualization")
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f'{model_name} - {attr_name} Fairness Analysis', fontsize=16, fontweight='bold')
    
    # 1. Demographic Parity (Selection Rate)
    selection_rates = [metrics_dict['selection_by_group'][g] for g in groups]
    axes[0, 0].bar(groups, selection_rates, color='steelblue', alpha=0.8)
    axes[0, 0].set_ylabel('Selection Rate', fontsize=11)
    axes[0, 0].set_title(f'Demographic Parity\n(Diff: {metrics_dict["demographic_parity_difference"]:.4f})', 
                        fontsize=11, fontweight='bold')
    axes[0, 0].grid(axis='y', alpha=0.3)
    for i, (g, rate) in enumerate(zip(groups, selection_rates)):
        axes[0, 0].text(i, rate, f'{rate:.2%}', ha='center', va='bottom', fontweight='bold')
    
    # 2. Equalized Odds - TPR
    tpr_values = [metrics_dict['tpr_by_group'][g] for g in groups]
    axes[0, 1].bar(groups, tpr_values, color='forestgreen', alpha=0.8)
    axes[0, 1].set_ylabel('True Positive Rate', fontsize=11)
    axes[0, 1].set_title(f'Equalized Odds - TPR\n(Diff: {metrics_dict["tpr_difference"]:.4f})', 
                        fontsize=11, fontweight='bold')
    axes[0, 1].grid(axis='y', alpha=0.3)
    for i, (g, tpr) in enumerate(zip(groups, tpr_values)):
        axes[0, 1].text(i, tpr, f'{tpr:.2%}', ha='center', va='bottom', fontweight='bold')
    
    # 3. Equalized Odds - FPR
    fpr_values = [metrics_dict['fpr_by_group'][g] for g in groups]
    axes[0, 2].bar(groups, fpr_values, color='coral', alpha=0.8)
    axes[0, 2].set_ylabel('False Positive Rate', fontsize=11)
    axes[0, 2].set_title(f'Equalized Odds - FPR\n(Diff: {metrics_dict["fpr_difference"]:.4f})', 
                        fontsize=11, fontweight='bold')
    axes[0, 2].grid(axis='y', alpha=0.3)
    for i, (g, fpr) in enumerate(zip(groups, fpr_values)):
        axes[0, 2].text(i, fpr, f'{fpr:.2%}', ha='center', va='bottom', fontweight='bold')
    
    # 4. Precision Parity
    precision_values = [metrics_dict['precision_by_group'][g] for g in groups]
    axes[1, 0].bar(groups, precision_values, color='purple', alpha=0.8)
    axes[1, 0].set_ylabel('Precision', fontsize=11)
    axes[1, 0].set_title(f'Precision Parity\n(Diff: {metrics_dict["precision_difference"]:.4f})', 
                        fontsize=11, fontweight='bold')
    axes[1, 0].grid(axis='y', alpha=0.3)
    for i, (g, prec) in enumerate(zip(groups, precision_values)):
        axes[1, 0].text(i, prec, f'{prec:.2%}', ha='center', va='bottom', fontweight='bold')
    
    # 5. Accuracy by Group
    accuracy_values = [by_group.loc[g, 'accuracy'] for g in groups]
    axes[1, 1].bar(groups, accuracy_values, color='teal', alpha=0.8)
    axes[1, 1].set_ylabel('Accuracy', fontsize=11)
    axes[1, 1].set_title('Accuracy by Group', fontsize=11, fontweight='bold')
    axes[1, 1].grid(axis='y', alpha=0.3)
    for i, (g, acc) in enumerate(zip(groups, accuracy_values)):
        axes[1, 1].text(i, acc, f'{acc:.2%}', ha='center', va='bottom', fontweight='bold')
    
    # 6. F1-Score by Group
    f1_values = [by_group.loc[g, 'f1'] for g in groups]
    axes[1, 2].bar(groups, f1_values, color='orange', alpha=0.8)
    axes[1, 2].set_ylabel('F1-Score', fontsize=11)
    axes[1, 2].set_title('F1-Score by Group', fontsize=11, fontweight='bold')
    axes[1, 2].grid(axis='y', alpha=0.3)
    for i, (g, f1) in enumerate(zip(groups, f1_values)):
        axes[1, 2].text(i, f1, f'{f1:.2%}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()


print("✓ Helper functions defined")


## 3. Comprehensive Fairness Analysis

Analyze both models across all protected attributes.


In [ ]:
# Store all fairness results
all_fairness_results = {}

# Models to analyze
models = {
    'Logistic Regression': (lr_test_pred, lr_test_proba),
    'Random Forest': (rf_test_pred, rf_test_proba)
}

print("=" * 60)
print("COMPREHENSIVE FAIRNESS ANALYSIS")
print("=" * 60)

# Analyze each model and each protected attribute
for model_name, (y_pred, y_proba) in models.items():
    print(f"\n{'='*60}")
    print(f"Analyzing: {model_name}")
    print(f"{'='*60}")
    
    all_fairness_results[model_name] = {}
    
    for attr in protected_attributes_list:
        if attr not in protected_test.columns:
            print(f"⚠ Warning: {attr} not found in protected_test, skipping...")
            continue
            
        sensitive_feature = protected_test[attr]
        
        # Skip if all values are the same
        if len(sensitive_feature.unique()) < 2:
            print(f"⚠ Warning: {attr} has only one unique value, skipping...")
            continue
        
        print(f"\n📊 Analyzing protected attribute: {attr}")
        print(f"   Groups: {sorted(sensitive_feature.unique())}")
        
        # Calculate fairness metrics
        metrics = calculate_fairness_metrics(
            y_test, y_pred, sensitive_feature, model_name, attr
        )
        
        all_fairness_results[model_name][attr] = metrics
        
        # Print summary
        print(f"\n   ⚖️  Fairness Metrics:")
        print(f"      Demographic Parity Difference: {metrics['demographic_parity_difference']:.4f}")
        print(f"      Equalized Odds Difference: {metrics['equalized_odds_difference']:.4f}")
        print(f"      Precision Difference: {metrics['precision_difference']:.4f}")
        print(f"      TPR Difference: {metrics['tpr_difference']:.4f}")
        print(f"      FPR Difference: {metrics['fpr_difference']:.4f}")
        
        # Check for significant disparities
        warnings = []
        if metrics['demographic_parity_difference'] > 0.05:
            warnings.append(f"⚠ Demographic parity disparity > 5pp")
        if metrics['equalized_odds_difference'] > 0.05:
            warnings.append(f"⚠ Equalized odds disparity > 5pp")
        if metrics['precision_difference'] > 0.05:
            warnings.append(f"⚠ Precision disparity > 5pp")
        
        if warnings:
            print(f"\n   {' '.join(warnings)}")
        
        # Display group-level metrics
        print(f"\n   📈 Group-Level Performance:")
        display(metrics['by_group'])
        
        # Visualize
        visualize_fairness_metrics(metrics, model_name, attr)

print("\n" + "=" * 60)
print("✅ COMPREHENSIVE ANALYSIS COMPLETE")
print("=" * 60)


## 4. Summary Comparison

Create summary tables comparing fairness metrics across models and attributes.


In [ ]:
# Create comprehensive summary table
summary_rows = []

for model_name in all_fairness_results:
    for attr_name in all_fairness_results[model_name]:
        metrics = all_fairness_results[model_name][attr_name]
        summary_rows.append({
            'Model': model_name,
            'Protected Attribute': attr_name,
            'Demographic Parity Diff': metrics['demographic_parity_difference'],
            'Equalized Odds Diff': metrics['equalized_odds_difference'],
            'Precision Diff': metrics['precision_difference'],
            'TPR Diff': metrics['tpr_difference'],
            'FPR Diff': metrics['fpr_difference']
        })

summary_df = pd.DataFrame(summary_rows)

print("=" * 80)
print("FAIRNESS METRICS SUMMARY")
print("=" * 80)
print("\nAll differences are absolute differences between groups.")
print("Lower values indicate better fairness.\n")
display(summary_df.round(4))

# Create heatmap of fairness disparities
# Check if we have enough data for heatmaps
num_models = summary_df['Model'].nunique()
num_attributes = summary_df['Protected Attribute'].nunique()

if num_models >= 2 and num_attributes >= 2:
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Pivot tables for heatmaps
    dp_pivot = summary_df.pivot(index='Protected Attribute', columns='Model', values='Demographic Parity Diff')
    eo_pivot = summary_df.pivot(index='Protected Attribute', columns='Model', values='Equalized Odds Diff')
    prec_pivot = summary_df.pivot(index='Protected Attribute', columns='Model', values='Precision Diff')
    
    # Demographic Parity Heatmap
    sns.heatmap(dp_pivot, annot=True, fmt='.4f', cmap='Reds', ax=axes[0], cbar_kws={'label': 'Difference'})
    axes[0].set_title('Demographic Parity Difference', fontsize=12, fontweight='bold')
    
    # Equalized Odds Heatmap
    sns.heatmap(eo_pivot, annot=True, fmt='.4f', cmap='Oranges', ax=axes[1], cbar_kws={'label': 'Difference'})
    axes[1].set_title('Equalized Odds Difference', fontsize=12, fontweight='bold')
    
    # Precision Parity Heatmap
    sns.heatmap(prec_pivot, annot=True, fmt='.4f', cmap='Purples', ax=axes[2], cbar_kws={'label': 'Difference'})
    axes[2].set_title('Precision Parity Difference', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠ Cannot create heatmaps: Need at least 2 models and 2 attributes")
    print(f"   Current: {num_models} model(s), {num_attributes} attribute(s)")
    print("   Displaying summary table instead (see above)")

print("\n📊 Key Findings:")
print(f"   • Highest Demographic Parity Disparity: {summary_df['Demographic Parity Diff'].max():.4f}")
print(f"   • Highest Equalized Odds Disparity: {summary_df['Equalized Odds Diff'].max():.4f}")
print(f"   • Highest Precision Disparity: {summary_df['Precision Diff'].max():.4f}")


## 5. Save Fairness Analysis Results

Save all fairness metrics for downstream comparison with mitigated models.


In [ ]:
# Prepare results for saving (convert to JSON-serializable format)
fairness_results_serializable = {}

for model_name in all_fairness_results:
    fairness_results_serializable[model_name] = {}
    for attr_name in all_fairness_results[model_name]:
        metrics = all_fairness_results[model_name][attr_name]
        
        # Convert by_group DataFrame to dict
        by_group_dict = metrics['by_group'].to_dict('index')
        
        fairness_results_serializable[model_name][attr_name] = {
            'demographic_parity_difference': float(metrics['demographic_parity_difference']),
            'demographic_parity_ratio': float(metrics['demographic_parity_ratio']),
            'equalized_odds_difference': float(metrics['equalized_odds_difference']),
            'equalized_odds_ratio': float(metrics['equalized_odds_ratio']),
            'precision_difference': float(metrics['precision_difference']),
            'precision_ratio': float(metrics['precision_ratio']),
            'tpr_difference': float(metrics['tpr_difference']),
            'fpr_difference': float(metrics['fpr_difference']),
            'by_group': by_group_dict,
            'selection_by_group': {str(k): float(v) for k, v in metrics['selection_by_group'].items()},
            'tpr_by_group': {str(k): float(v) for k, v in metrics['tpr_by_group'].items()},
            'fpr_by_group': {str(k): float(v) for k, v in metrics['fpr_by_group'].items()},
            'precision_by_group': {str(k): float(v) for k, v in metrics['precision_by_group'].items()},
            'group_sizes': {str(k): int(v) for k, v in metrics.get('group_sizes', {}).items()}
        }

# Save to JSON
with open(results_dir / 'baseline_fairness_analysis.json', 'w') as f:
    json.dump(fairness_results_serializable, f, indent=2)

# Save summary table as CSV
summary_df.to_csv(results_dir / 'baseline_fairness_summary.csv', index=False)

print("✅ Fairness analysis results saved!")
print(f"   • baseline_fairness_analysis.json")
print(f"   • baseline_fairness_summary.csv")
print(f"\n   Saved to: {results_dir.absolute()}")

print("\n" + "=" * 60)
print("NOTEBOOK COMPLETE!")
print("=" * 60)
print("Next steps:")
print("  → Proceed to notebook 04_mitigation.ipynb")
print("  → Apply fairness-aware techniques to reduce identified disparities")
